## Record Linkage del dataset

In [1]:
import pandas as pd
import numpy as np
import time

import recordlinkage
import warnings
from recordlinkage.index import Full

warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('schemaAlignment/aziende_merged_finale.csv', low_memory=False)
df = df.drop(['Unnamed: 0'], axis=1)
df

,name,company_website,country,number_of_employees,industry,founded_year,market_cap,revenue,rank,sector,city,valuation,stock,ceo,market_value
0,Proteus Digital Health Careers,www.proteus.com,"Redwood City, CA",201 to 500 employees,Pharmaceuticals,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Zelis Careers,www.zelis.com,"Bedminster, NJ","1,001 to 5,000 employees",Information Technology & Services,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UpNest Careers,www.upnest.com,"Burlingame, CA",1 to 50 employees,Real Estate,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,conferacity Careers,conferacity.com,"Menlo Park, CA",1 to 50 employees,Broadcast & Online Media,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Zenoti Careers,www.zenoti.com,"Bellevue, WA","501 to 1,000 employees",Computer Software,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120254,Wayne's Coffee,NaN,NaN,NaN,Consumer services,NaN,NaN,NaN,NaN,Restaurants & bars,NaN,NaN,NaN,NaN,NaN
120255,WESC,NaN,NaN,NaN,Consumer goods,NaN,NaN,NaN,NaN,Clothing & accessories,NaN,NaN,NaN,NaN,NaN
120256,WG Film,NaN,NaN,NaN,Consumer services,NaN,NaN,NaN,NaN,Broadcasting & entertainment,NaN,NaN,NaN,NaN,NaN
120257,WM-data,NaN,NaN,NaN,Technology,NaN,NaN,NaN,NaN,Software,NaN,NaN,NaN,NaN,NaN


In [3]:
# Indicizzazione dei dati e calcolo tempo necessario per indicizzazione
start_time = time.time()

indexer = recordlinkage.Index()
indexer.block('name') # Blocco sulla colonna "name" (blocco informazioni utilizzate per l'indicizzazione)
candidate_links = indexer.index(df)

print("Tempo di indicizzazione dei dati: ", time.time() - start_time, "secondi")
print(candidate_links)

Tempo di indicizzazione dei dati:  0.16800212860107422 secondi
MultiIndex([( 51039,    952),
            ( 25131,    953),
            ( 47097,    953),
            ( 47097,  25131),
            ( 95494,    953),
            ( 95494,  25131),
            ( 95494,  47097),
            (100561,    953),
            (100561,  25131),
            (100561,  47097),
            ...
            (118747, 117706),
            (117768, 117732),
            (118347, 117732),
            (118347, 117768),
            (119764, 117776),
            (117935, 117812),
            (119467, 117821),
            (118118, 117923),
            (118021, 117943),
            (118745, 118013)],
           length=155900)


In [4]:
# Confrontare ogni coppia di record candidate e calcolo empo necessario per effettuare il confronto
start_time = time.time()

compare_cl = recordlinkage.Compare()

compare_cl.string('name', 'name', label='name', threshold = 0.9)
compare_cl.string('country', 'country', label='country', threshold = 0.8)
compare_cl.string('company_website', 'company_website', label='comprany_website', threshold = 0.9)
compare_cl.string('number_of_employees', 'number_of_employees', label='number_of_employees', threshold = 0.6)
compare_cl.string('industry', 'industry', label='industry', threshold = 0.7)
compare_cl.string('market_cap', 'market_cap', label='market_cap')
compare_cl.string('revenue', 'revenue', label='revenue')
compare_cl.string('rank', 'rank', label='rank')
compare_cl.string('sector', 'sector', label='sector')
compare_cl.string('city', 'city', label='city')
compare_cl.string('valuation', 'valuation', label='valuation')
compare_cl.string('stock', 'stock', label='stock')
compare_cl.string('ceo', 'ceo', label='ceo')
compare_cl.string('market_value', 'market_value', label='market_value')
compare_cl.exact('founded_year', 'founded_year', label='founded_year')

features = compare_cl.compute(candidate_links, df)

print("Tempo di confronto delle coppie di record: ", time.time() - start_time, "secondi")

Tempo di confronto delle coppie di record:  10.900238990783691 secondi


In [5]:
features = features.head(len(df))
features

name  country  comprany_website  number_of_employees  industry  \
51039 952     1.0      0.0               0.0                  0.0       0.0   
25131 953     1.0      0.0               0.0                  0.0       0.0   
47097 953     1.0      0.0               0.0                  0.0       0.0   
      25131   1.0      0.0               0.0                  0.0       0.0   
95494 953     1.0      1.0               0.0                  1.0       0.0   
...           ...      ...               ...                  ...       ...   
81476 81474   1.0      1.0               0.0                  0.0       0.0   
      81475   1.0      1.0               0.0                  0.0       0.0   
81483 81482   1.0      0.0               0.0                  0.0       0.0   
81484 81482   1.0      0.0               0.0                  0.0       0.0   
      81483   1.0      0.0               0.0                  0.0       0.0   

             market_cap   revenue      rank  sector  city  valuation  stock  \
51039 952      0.125000  0.000000  0.000000     0.0   0.0   0.000000    0.0   
25131 953      0.000000  0.000000  0.000000     0.0   0.0   0.000000    0.0   
47097 953      0.230769  0.000000  0.000000     0.0   0.0   0.000000    0.0   
      25131    0.000000  0.000000  0.000000     0.0   0.0   0.000000    0.0   
95494 953      0.571429  0.142857  0.000000     0.0   0.0   0.000000    0.0   
...                 ...       ...       ...     ...   ...        ...    ...   
81476 81474    0.857143  0.000000  0.833333     0.0   0.0   0.333333    0.0   
      81475    0.000000  0.000000  0.000000     0.0   0.0   0.000000    0.0   
81483 81482    0.857143  0.000000  0.000000     0.0   0.0   0.666667    0.0   
81484 81482    0.857143  0.000000  0.666667     0.0   0.0   0.666667    0.0   
      81483    0.857143  0.000000  0.000000     0.0   0.0   0.666667    0.0   

             ceo  market_value  founded_year  
51039 952    0.0           0.0             0  
25131 953    0.0           0.0             0  
47097 953    0.0           0.0             0  
      25131  0.0           0.0             0  
95494 953    0.0           0.0             0  
...          ...           ...           ...  
81476 81474  0.0           0.0             0  
      81475  0.0           0.0             0  
81483 81482  0.0           0.0             0  
81484 81482  0.0           0.0             0  
      81483  0.0           0.0             0  

[120259 rows x 15 columns]

### **Dedupe**

In [1]:
import pandas as pd
import dedupe
import time

df = pd.read_csv('aziendeMerged_ordinato.csv', low_memory=False)

In [ ]:
# Conta il numero di valori nulli in ciascuna colonna
num_missing = df.isnull().sum()
num_missing = num_missing.sort_values()

print(num_missing)

In [ ]:
df = df.loc[df.isna().sum(axis=1) <= 7]
df

In [ ]:
# Conta il numero di valori nulli in ciascuna colonna
num_missing = df.isnull().sum()
num_missing = num_missing.sort_values()

print(num_missing)

In [ ]:
df = df.astype(str)

In [2]:
# crea un dizionario dei campi del dataset da usare per la deduplicazione
fields = [
        {'field': 'name', 'type': 'String'},
        {'field': 'country', 'type': 'String', 'has missing': True},
        {'field': 'number_of_employees', 'type': 'String', 'has missing': True},
        {'field': 'founded_year', 'type': 'String', 'has missing': True},
        {'field': 'revenue', 'type': 'String', 'has missing': True},
    ]
deduper = dedupe.Dedupe(fields, recall=0.5)

In [ ]:
data = df.fillna('').to_dict(orient='records')

In [3]:
for record in data:
    for field in fields:
        if field['field'] in record and not isinstance(record[field['field']], (str, bytes)):
            print(f"Errore nella riga {record}: il valore '{record[field['field']]}' nella colonna '{field['field']}' non è di tipo stringa o bytes.")


In [4]:
# Clustering:
data_dim = data[:1000]
data_d = {}
for i, record in enumerate(data_dim):
    data_d[i] = record


training_data = deduper.prepare_training(data_d)

In [ ]:
# inizio del training dell'algoritmo di deduplicazione
dedupe.console_label(deduper)

In [ ]:
deduper.train(training_data) # ERRORE

In [ ]:
start_time = time.time()
clustered_dupes = deduper.partition(data_d, 0.5)

print('Numero sets duplicati', len(clustered_dupes))
print("Durata clustering: ", time.time() - start_time, "secondi")

In [ ]:
import dedupe.convenience

# Calcola le metriche di valutazione
# Prende come input il set di record (data_d) e i cluster risultanti dalla deduplicazione (clustered_dupes). 
# La funzione restituisce una tupla contenente le metriche di valutazione richieste.
evaluation = dedupe.convenience.evaluateDuplicates(data_d, clustered_dupes)

# Estrae le metriche di interesse
precision = evaluation[0]
recall = evaluation[1]
f_measure = evaluation[2]

print('Precision: ', precision)
print('Recall: ', recall)
print('F-measure: ', f_measure)